
# Linux umask (权限掩码) 以及 Python os.umask 的使用

## I. Linux umask
[参考](https://blog.csdn.net/JesseYoung/article/details/42525341)

### 1. 简介 
　　linux中的 umask 函数主要用于：在创建新文件或目录时 屏蔽掉新文件或目录不应有的访问允许权限。
　　文件的访问允许权限共有9种，分别是：rwxrwxrwx
　　它们分别代表：用户读 用户写 用户执行　组读 组写 组执行 其它读 其它写 其它执行

### 2. 功能及参数介绍
　　umask（显示、设置文件的缺省权限即默认权限） 
　　语法： umask [-S] [权限掩码]
　　补充说明： umask 可用来设定[权限掩码]。[权限掩码]是由 3 个八进制的数字所组成，将现有的存取权限减掉权限掩码后，即可产生建立文件时预设的权限。
　　-S 以 rwx 形式显示（Show）新建文件或目录缺省权限 

　　例1： 查看默认权限（一般是 0022） 
```shell
umask
``` 
　　注：
　　0 - 特殊权限位  
　　022 - 用户权限位，权限掩码值。则实际权限值：777 - 022 = 755（目录权限）  

　　例2： 改变系统缺省的创建文件的权限（目录默认权限变为750，文件为640，一并设置）
　　umask 027
　　注：
　　Linux权限规则：缺省创建的文件不能授予可执行x权限（文件默认权限644） 这规则屏蔽了很多的攻击和病毒，任何木马病毒不可执行就没价值。 
    
　　文件：用八进制基数666，即无x位（可执行位）rw- rw- rw- 。执行位需由用户自行加入。
　　例一：设要生成的文件以rw- r-- r--这样的权限字出现，即真实权限用八进制表示为644，则被666基数减得022，022即掩码。使用umask 022。
　　注：033效果与022一样，假设使用033掩码进??行设置，则真实权限应为633即rw- rx rx ，但前提规定文件不生成x位，所以文件的权限最终将以rw-r--r --出现。

　　目录：用八进制基数777
　　例二：设要生成的目录权限以rwxr-xr-x这样的权限字出现，即真实权限用八进制表示为755，则被基数为777的权限字相减后，得掩码022。则使用umask 022进行设置。

## 3. 总结 
　　掌握二个要点  
　　1. 文件基数为 666，目录为 777，即文件无设 x 位，目录可设 x 位。  
　　2. chmod 是设哪个位，哪么哪个位就有权限，而 umask 是设哪个位，则哪个位上就没权限，即 chmod 指令是“给于”使用或者是将要改变的许可权限，而 umask 则是“取消”使用许可权限。  

## II. Python os.umask
[参考](https://stackoverflow.com/questions/10291131/how-to-use-os-umask-in-python?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa)
>Misunderstanding of umask, I think. The umask sets the default denials, not the default permissions. So
>```python
import os
oldmask = os.umask (022)  # 设置当前进程（或子进程）权限掩码（暂时）为 022，并返回原来的权限掩码，保存以供恢复
fh1 = os.open ("qq1.junk", os.O_CREAT, 0777)  # 创建一个权限为 0777 的文件，在权限掩码 0022 的作用下，实际权限为 777-022=755
fh2 = os.open ("qq2.junk", os.O_CREAT, 0022)  # 同理，实际权限为 000 
os.umask (oldmask)  # 恢复
os.close (fh1)
os.close (fh2)
```
should indeed produce files as follows:
```bash
-rwxr-xr-x 1 pax pax 0 Apr 24 11:11 qq1.junk
---------- 1 pax pax 0 Apr 24 11:11 qq2.junk
```
The umask 022 removes write access for group and others, which is exactly the behaviour we see there. I find it helps to go back to the binary that the octal numbers represent:
>```bash
 usr grp others 
-rwx rwx rwx is represented in octal as 0777, requested for qq1.junk
-000 010 010 umask of 022 removes any permission where there is a 1
-rwx r-x r-x is the result achieved requesting 0777 with umask of 022
---- -w- -w- is represented in octal as 0022, requested for qq2.junk
-000 010 010 umask of 022 removes any permission where there is a 1
---- --- --- is the result achieved requesting 0022 with umask of 022
```
The program is behaving as you asked it to, not necessarily as you thought it should. Common situation, that, with computers :-)